# Change Detection (CD) with Random Forest (RF)

### Post-Classification CD using RF classifier
### Using Google Earth Engine Python API and NICFI Normalized Analytic Basemap from December 2015, December 2017, December 2019, and December 2021

Author: Finn Geiger\
Date: April 6th 2023\
Contact:
- https://github.com/finn-geiger
- https://www.linkedin.com/in/finn-geiger-b1329a20b/

### Steps:
1. Importing the datasets and classification result of 2015
2. OPIT classification for 2017, 2019, and 2021
3. CD for TOI 2015 to 2017
    1. performing CD
    2. Two-Step Accuracy Assessment
    3. Creating map products for CD
4. CD for TOI 2017 to 2019
    1. performing CD
    2. Two-Step Accuracy Assessment
    3. Creating map products for CD
5. CD for TOI 2019 to 2021
    1. performing CD
    2. Two-Step Accuracy Assessment
    3. Creating map products for CD
6. Final products

### 1 Import and setup
#### 1.1 Importing the required libraries and packages

In [60]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import geemap
import ee
import os
import time
import pandas as pd
from tabulate import tabulate
#%pip install tabulate


The following classes and landcover IDs will be used:

In [61]:
info = {'Class name': ['Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies'],
        'landcover ID': [1, 2, 3, 4, 5, 6, 7]}

print(tabulate(info, headers='keys', tablefmt='fancy_grid'))

╒══════════════╤════════════════╕
│ Class name   │   landcover ID │
╞══════════════╪════════════════╡
│ Informal     │              1 │
├──────────────┼────────────────┤
│ Formal       │              2 │
├──────────────┼────────────────┤
│ Industrial   │              3 │
├──────────────┼────────────────┤
│ Roads        │              4 │
├──────────────┼────────────────┤
│ Vacant land  │              5 │
├──────────────┼────────────────┤
│ Vegetation   │              6 │
├──────────────┼────────────────┤
│ Water-bodies │              7 │
╘══════════════╧════════════════╛


##### When first using the GEE Python API the user must authenticate and initialize the environment by using the following two lines of codes:

In [62]:
#ee.Authenticate() 
#ee.Initialize()

In [63]:
# creating the map
Map = geemap.Map()

# loading the interactive map
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

#### 1.2 Importing the datasets from GEE assets and data catalog and clipping the basemap to the AOI

In [64]:
# Loading the Base scene
nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa')

# Filter basemaps by date and get the first image from filtered results
basemap_2017_12 = nicfi.filter(ee.Filter.date('2017-12-01','2018-01-01')).first()
basemap_2019_12 = nicfi.filter(ee.Filter.date('2019-12-01','2029-01-01')).first()
basemap_2021_12 = nicfi.filter(ee.Filter.date('2021-12-01','2022-01-01')).first()

# importing the classified scene from 2015
classified_TOI1_2015 = ee.Image('users/s85315/masterthesis/change_detection_results/classified_TOI1_RF')

# Visualizing the scene
vis_params = {"bands":["R","G","B"],"min":64,"max":5454,"gamma":1.8}

# Adding the basemap to the map
Map.centerObject(basemap_2017_12, 4)
# Map.addLayer(basemap_2017_12, vis_params, '2017-12 mosaic', False)
# Map.addLayer(basemap_2019_12, vis_params, '2019-12 mosaic', False)
# Map.addLayer(basemap_2021_12, vis_params, '2021-12 mosaic', False)

In [65]:
# Loading the AOI and Masking the base scene
vis_params_aoi = {'color': 'blue'}
aoi_windhoek = ee.FeatureCollection('users/s85315/masterthesis/Study_Area_Windhoek')

# Adding the AOI to the map
Map.addLayer(aoi_windhoek, vis_params_aoi, 'AOI')
Map.centerObject(aoi_windhoek, 12)

In [66]:
# clipping the clipped_TOI1_2015 to the AOI
clipped_TOI2_2017 = basemap_2017_12.clipToCollection(aoi_windhoek)
clipped_TOI3_2019 = basemap_2019_12.clipToCollection(aoi_windhoek)
clipped_TOI4_2021 = basemap_2021_12.clipToCollection(aoi_windhoek)

Map.addLayer(clipped_TOI2_2017, vis_params, 'clipped_2017', False)
Map.addLayer(clipped_TOI3_2019, vis_params, 'clipped_2019', False)
Map.addLayer(clipped_TOI4_2021, vis_params, 'clipped_2021', False)

In [67]:
# importing training data samples and adding them to the map
# Transforming the Geometries into FeatureCollections and applying properties
training_samples_2017 = ee.FeatureCollection('users/s85315/masterthesis/TrainingSamples/ChangeDetection/TS_TOI2_2017_RPoints')
training_samples_2019 = ee.FeatureCollection('users/s85315/masterthesis/TrainingSamples/ChangeDetection/TS_TOI3_2019_RPoints')
training_samples_2021 = ee.FeatureCollection('users/s85315/masterthesis/TrainingSamples/ChangeDetection/TS_TOI4_2021_RPoints')

# adding the layers to the map
# Map.addLayer(training_samples_2017, {}, 'Training Samples for 2017')
# Map.addLayer(training_samples_2019, {}, 'Training Samples for 2019')
# Map.addLayer(training_samples_2021, {}, 'Training Samples for 2021')

### 2 OPIT Classification with RF for TOIs 2017, 2019, and 2021

#### 2.1 Configuration and creation of the empty RF classifier

In [68]:
# creating variable for parameter settings
# Mtry will be set to default
RF_param = 550

classifier_params = {
              'numberOfTrees':RF_param, # 	Ntree; The number of decision trees to create.
              'variablesPerSplit':None, # Mtry; The number of variables per split. If unspecified, uses the square root of the number of variables.
              'minLeafPopulation':1, # smallest sample size possible per leaf
              'bagFraction':0.5, #The fraction of input to bag per tree.
              'maxNodes':None, # the number of individual decision tree models
              'seed': 0}  # The randomization seed.

#### 2.2 Applying training samples on the scenes

##### 2.2.1 TOI2 - 2017

In [69]:
# adding the training samples to the basescene
training_2017 = clipped_TOI2_2017.sampleRegions(**{
    'collection': training_samples_2017, 
    'properties': ['landcover'], 
    'scale': 4.77
})

# creating the classifier using RF
classifier_2017 = ee.Classifier.smileRandomForest(**classifier_params).train(**{
  'features': training_2017,  
  'classProperty': 'landcover', 
  'inputProperties': clipped_TOI2_2017.bandNames()
})

##### 2.2.2 TOI3 - 2019

In [70]:
# adding the training samples to the basescene
training_2019 = clipped_TOI3_2019.sampleRegions(**{
    'collection': training_samples_2019, 
    'properties': ['landcover'], 
    'scale': 4.77
})

# creating the classifier using RF
classifier_2019 = ee.Classifier.smileRandomForest(**classifier_params).train(**{
  'features': training_2019,  
  'classProperty': 'landcover', 
  'inputProperties': clipped_TOI3_2019.bandNames()
})

##### 2.2.3 TOI4 - 2021

In [71]:
# adding the training samples to the basescene
training_2021 = clipped_TOI4_2021.sampleRegions(**{
    'collection': training_samples_2021, 
    'properties': ['landcover'], 
    'scale': 4.77
})

# creating the classifier using RF
classifier_2021 = ee.Classifier.smileRandomForest(**classifier_params).train(**{
  'features': training_2021,  
  'classProperty': 'landcover', 
  'inputProperties': clipped_TOI4_2021.bandNames()
})

#### 2.3 classifying the basescene and visualizing the product

In [72]:
# classifying the basescene using the created classifier
classified_TOI2_2017 = clipped_TOI2_2017.classify(classifier_2017)
classified_TOI3_2019 = clipped_TOI3_2019.classify(classifier_2019)
classified_TOI4_2021 = clipped_TOI4_2021.classify(classifier_2021)

# creating the visualization parameters
palette = ['c43c39', 'e5b636', '2f2f2f', 'aaaaaa', 'b08e7a', '85b66f', 'a5bfdd']
vis_params_classified = {'min': 1, 'max': 7, 'palette': palette}


# Map.addLayer(classified_TOI1_2015, vis_params_classified, 'classified scene - 2015')
# Map.addLayer(classified_TOI2_2017, vis_params_classified, 'classified scene - 2017')
# Map.addLayer(classified_TOI3_2019, vis_params_classified, 'classified scene - 2019')
# Map.addLayer(classified_TOI4_2021, vis_params_classified, 'classified scene - 2021')


#### 2.4 Exporting the results

##### 2.4.1 Exporting to Google Drive

In [82]:
# converting the FeatureCollection to Geometry for export
aoi_geom = aoi_windhoek.geometry()

# # exporting to Google drive with GEE API
# export_TOI2 = ee.batch.Export.image.toDrive(**{
#     'image': classified_TOI2_2017,
#     'description': 'classified_TOI2_RF', # TODO: change name here
#     'folder': 'masterthesis/change_detection_results', # TODO: change name here
#     'scale': 4.77,
#     'region': aoi_geom
# })

# # starting the process
# export_TOI2.start()

# # tracking the process
# while export_TOI2.active():
#   print('Polling for task (id: {}).'.format(export_TOI2.id))
#   time.sleep(5)

In [74]:
# # exporting to Google drive with GEE API
# export_TOI3 = ee.batch.Export.image.toDrive(**{
#     'image': classified_TOI3_2019,
#     'description': 'classified_TOI3_RF', # TODO: change name here
#     'folder': 'masterthesis/change_detection_results', # TODO: change name here
#     'scale': 4.77,
#     'crs': 'EPSG:32733',
#     'region': aoi_geom
# })

# # starting the process
# export_TOI3.start()

# # tracking the process
# while export_TOI3.active():
#   print('Polling for task (id: {}).'.format(export_TOI3.id))
#   time.sleep(5)

Polling for task (id: PXZXO4WQ5ZB2JSNW3YGPOPUY).
Polling for task (id: PXZXO4WQ5ZB2JSNW3YGPOPUY).
Polling for task (id: PXZXO4WQ5ZB2JSNW3YGPOPUY).
Polling for task (id: PXZXO4WQ5ZB2JSNW3YGPOPUY).
Polling for task (id: PXZXO4WQ5ZB2JSNW3YGPOPUY).
Polling for task (id: PXZXO4WQ5ZB2JSNW3YGPOPUY).
Polling for task (id: PXZXO4WQ5ZB2JSNW3YGPOPUY).
Polling for task (id: PXZXO4WQ5ZB2JSNW3YGPOPUY).
Polling for task (id: PXZXO4WQ5ZB2JSNW3YGPOPUY).
Polling for task (id: PXZXO4WQ5ZB2JSNW3YGPOPUY).
Polling for task (id: PXZXO4WQ5ZB2JSNW3YGPOPUY).
Polling for task (id: PXZXO4WQ5ZB2JSNW3YGPOPUY).
Polling for task (id: PXZXO4WQ5ZB2JSNW3YGPOPUY).
Polling for task (id: PXZXO4WQ5ZB2JSNW3YGPOPUY).
Polling for task (id: PXZXO4WQ5ZB2JSNW3YGPOPUY).
Polling for task (id: PXZXO4WQ5ZB2JSNW3YGPOPUY).
Polling for task (id: PXZXO4WQ5ZB2JSNW3YGPOPUY).
Polling for task (id: PXZXO4WQ5ZB2JSNW3YGPOPUY).
Polling for task (id: PXZXO4WQ5ZB2JSNW3YGPOPUY).
Polling for task (id: PXZXO4WQ5ZB2JSNW3YGPOPUY).
Polling for task (id

In [75]:
# # exporting to Google drive with GEE API
# export_TOI4 = ee.batch.Export.image.toDrive(**{
#     'image': classified_TOI4_2021,
#     'description': 'classified_TOI4_RF', # TODO: change name here
#     'folder': 'masterthesis/change_detection_results', # TODO: change name here
#     'scale': 4.77,
#     'region': aoi_geom
# })

# # starting the process
# export_TOI4.start()

# # tracking the process
# while export_TOI4.active():
#   print('Polling for task (id: {}).'.format(export_TOI4.id))
#   time.sleep(5)

##### 2.4.2 Exporting to Asset

In [76]:
# # exporting to Google Asset
# export_TOI2 = ee.batch.Export.image.toAsset(**{
#   'image': classified_TOI2_2017,
#   'description': 'Export classified map',
#   'assetId': 'users/s85315/masterthesis/change_detection_results/classified_TOI2_RF', # TODO: change name here
#   'scale': 4.77,
#   'region': aoi_geom
# })

# # starting the process
# export_TOI2.start()

# # tracking the process
# while export_TOI2.active():
#   print('Polling for task (id: {}).'.format(export_TOI2.id))
#   time.sleep(5)

Polling for task (id: YBJIYUDMG2D5DCYS4TRGNEZJ).
Polling for task (id: YBJIYUDMG2D5DCYS4TRGNEZJ).
Polling for task (id: YBJIYUDMG2D5DCYS4TRGNEZJ).
Polling for task (id: YBJIYUDMG2D5DCYS4TRGNEZJ).
Polling for task (id: YBJIYUDMG2D5DCYS4TRGNEZJ).
Polling for task (id: YBJIYUDMG2D5DCYS4TRGNEZJ).
Polling for task (id: YBJIYUDMG2D5DCYS4TRGNEZJ).
Polling for task (id: YBJIYUDMG2D5DCYS4TRGNEZJ).
Polling for task (id: YBJIYUDMG2D5DCYS4TRGNEZJ).
Polling for task (id: YBJIYUDMG2D5DCYS4TRGNEZJ).
Polling for task (id: YBJIYUDMG2D5DCYS4TRGNEZJ).
Polling for task (id: YBJIYUDMG2D5DCYS4TRGNEZJ).
Polling for task (id: YBJIYUDMG2D5DCYS4TRGNEZJ).
Polling for task (id: YBJIYUDMG2D5DCYS4TRGNEZJ).
Polling for task (id: YBJIYUDMG2D5DCYS4TRGNEZJ).
Polling for task (id: YBJIYUDMG2D5DCYS4TRGNEZJ).
Polling for task (id: YBJIYUDMG2D5DCYS4TRGNEZJ).
Polling for task (id: YBJIYUDMG2D5DCYS4TRGNEZJ).
Polling for task (id: YBJIYUDMG2D5DCYS4TRGNEZJ).
Polling for task (id: YBJIYUDMG2D5DCYS4TRGNEZJ).
Polling for task (id

In [77]:
# # exporting to Google Asset
# export_TOI3 = ee.batch.Export.image.toAsset(**{
#   'image': classified_TOI3_2019,
#   'description': 'Export classified map',
#   'assetId': 'users/s85315/masterthesis/change_detection_results/classified_TOI3_RF', # TODO: change name here
#   'scale': 4.77,
#   'crs': 'EPSG:32733',
#   'region': aoi_geom
# })

# # starting the process
# export_TOI3.start()

# # tracking the process
# while export_TOI3.active():
#   print('Polling for task (id: {}).'.format(export_TOI3.id))
#   time.sleep(5)

Polling for task (id: YJ57U4NAV2XV55N5O6DEZT6O).
Polling for task (id: YJ57U4NAV2XV55N5O6DEZT6O).
Polling for task (id: YJ57U4NAV2XV55N5O6DEZT6O).
Polling for task (id: YJ57U4NAV2XV55N5O6DEZT6O).
Polling for task (id: YJ57U4NAV2XV55N5O6DEZT6O).
Polling for task (id: YJ57U4NAV2XV55N5O6DEZT6O).
Polling for task (id: YJ57U4NAV2XV55N5O6DEZT6O).
Polling for task (id: YJ57U4NAV2XV55N5O6DEZT6O).
Polling for task (id: YJ57U4NAV2XV55N5O6DEZT6O).
Polling for task (id: YJ57U4NAV2XV55N5O6DEZT6O).
Polling for task (id: YJ57U4NAV2XV55N5O6DEZT6O).
Polling for task (id: YJ57U4NAV2XV55N5O6DEZT6O).
Polling for task (id: YJ57U4NAV2XV55N5O6DEZT6O).
Polling for task (id: YJ57U4NAV2XV55N5O6DEZT6O).
Polling for task (id: YJ57U4NAV2XV55N5O6DEZT6O).
Polling for task (id: YJ57U4NAV2XV55N5O6DEZT6O).
Polling for task (id: YJ57U4NAV2XV55N5O6DEZT6O).
Polling for task (id: YJ57U4NAV2XV55N5O6DEZT6O).
Polling for task (id: YJ57U4NAV2XV55N5O6DEZT6O).
Polling for task (id: YJ57U4NAV2XV55N5O6DEZT6O).
Polling for task (id

In [78]:
# # exporting to Google Asset
# export_TOI4 = ee.batch.Export.image.toAsset(**{
#   'image': classified_TOI4_2021,
#   'description': 'Export classified map',
#   'assetId': 'users/s85315/masterthesis/change_detection_results/classified_TOI4_RF', # TODO: change name here
#   'scale': 4.77,
#   'crs': 'EPSG:32733',
#   'region': aoi_geom
# })

# # starting the process
# export_TOI4.start()

# # tracking the process
# while export_TOI4.active():
#   print('Polling for task (id: {}).'.format(export_TOI4.id))
#   time.sleep(5)

Polling for task (id: 2C74JAH3ZIHLJBQWLPZUKFHI).
Polling for task (id: 2C74JAH3ZIHLJBQWLPZUKFHI).
Polling for task (id: 2C74JAH3ZIHLJBQWLPZUKFHI).
Polling for task (id: 2C74JAH3ZIHLJBQWLPZUKFHI).
Polling for task (id: 2C74JAH3ZIHLJBQWLPZUKFHI).
Polling for task (id: 2C74JAH3ZIHLJBQWLPZUKFHI).
Polling for task (id: 2C74JAH3ZIHLJBQWLPZUKFHI).
Polling for task (id: 2C74JAH3ZIHLJBQWLPZUKFHI).
Polling for task (id: 2C74JAH3ZIHLJBQWLPZUKFHI).
Polling for task (id: 2C74JAH3ZIHLJBQWLPZUKFHI).
Polling for task (id: 2C74JAH3ZIHLJBQWLPZUKFHI).
Polling for task (id: 2C74JAH3ZIHLJBQWLPZUKFHI).
Polling for task (id: 2C74JAH3ZIHLJBQWLPZUKFHI).
Polling for task (id: 2C74JAH3ZIHLJBQWLPZUKFHI).
Polling for task (id: 2C74JAH3ZIHLJBQWLPZUKFHI).
Polling for task (id: 2C74JAH3ZIHLJBQWLPZUKFHI).
Polling for task (id: 2C74JAH3ZIHLJBQWLPZUKFHI).
Polling for task (id: 2C74JAH3ZIHLJBQWLPZUKFHI).
Polling for task (id: 2C74JAH3ZIHLJBQWLPZUKFHI).
Polling for task (id: 2C74JAH3ZIHLJBQWLPZUKFHI).
Polling for task (id

### 3 CD for TOI 2015 to 2017

#### 3.1 Performing the CD analysis

In [79]:
changed_areas_2015_2017 = classified_TOI2_2017.subtract(classified_TOI1_2015).neq(0)
changed_areas_2017_2019 = classified_TOI3_2019.subtract(classified_TOI2_2017).neq(0)
changed_areas_2019_2021 = classified_TOI4_2021.subtract(classified_TOI3_2019).neq(0)
changed_areas_2015_2021 = classified_TOI4_2021.subtract(classified_TOI1_2015).neq(0)

In [81]:
vis_params_changed = {'min':0, 'max':1, 'palette': ['white', 'red']}


Map.addLayer(changed_areas_2015_2017, vis_params_changed, 'Changed areas 2015 to 2017')
Map.addLayer(changed_areas_2017_2019, vis_params_changed, 'Changed areas 2017 to 2019')
Map.addLayer(changed_areas_2019_2021, vis_params_changed, 'Changed areas 2019 to 2021')
Map.addLayer(changed_areas_2015_2021, vis_params_changed, 'Changed areas 2015 to 2021')

transition_2015_2017 = classified_TOI1_2015.multiply(100).add(classified_TOI2_2017).rename('transitions')
transition_2017_2019 = classified_TOI2_2017.multiply(100).add(classified_TOI3_2019).rename('transitions')
transition_2019_2021 = classified_TOI3_2019.multiply(100).add(classified_TOI4_2021).rename('transitions')
transition_2015_2021 = classified_TOI1_2015.multiply(100).add(classified_TOI4_2021).rename('transitions')



#### Calculation of transitions in Pixels

#### Transition for 2015 to 2017

In [84]:
# calculating the size per 
transition_matrix_CD1 = transition_2015_2017.reduceRegion(**{
  'reducer': ee.Reducer.frequencyHistogram(),
  'geometry': aoi_geom,
  'scale': 4.77,
  'tileScale': 16,
  'maxPixels': 1e10
  })

transitions_2015_2017 = transition_matrix_CD1.get('transitions').getInfo()
df_transitions_2015_2017 = pd.DataFrame(transitions_2015_2017, index=[0])
df_transitions_2015_2017_long = pd.melt(df_transitions_2015_2017, var_name='From-To', value_name="Pixel")
df_transitions_2015_2017_long['Pixel'] = df_transitions_2015_2017_long['Pixel'].astype(int)


print(df_transitions_2015_2017_long.head())
df_transitions_2015_2017.to_csv('./data/transitions_pixels_2015_2017.csv', index=False, sep=";")


  From-To   Pixel
0     101  183631
1     102   47427
2     103   44100
3     104  115034
4     105   63504


#### Calculating the transition values per class for 2015-2017

In [85]:
area_changed_CD1_sqm = ee.Image.pixelArea().addBands(transition_2015_2017)
area_changed_CD1_ha = ee.Image.pixelArea().divide(10000).addBands(transition_2015_2017)


# getting the values per class
areas_changed_CD1_sqm_class = area_changed_CD1_sqm.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
    'groupField': 1,
    'groupName': 'transitions',
    }),
    'geometry': aoi_geom, 
    'scale': 4.77,
    'crs': 'EPSG:32733'
    # 'tileScale': 16,
    # 'maxPixels': 1e10
    })

class_areas = ee.List(areas_changed_CD1_sqm_class.get('groups'))
print(class_areas.getInfo())

df_class_areas = pd.DataFrame(class_areas.getInfo())
df_class_areas['sum'] = df_class_areas['sum'].astype(int)
print(df_class_areas['sum'].sum())
df_class_areas.to_csv('./data/transitions_sqm_per_class_2015_2017.csv', sep=";", index=False)

[{'sum': 3511104.0778485243, 'transitions': 101}, {'sum': 929541.9076126025, 'transitions': 102}, {'sum': 846146.9000753141, 'transitions': 103}, {'sum': 2206405.300093789, 'transitions': 104}, {'sum': 1252361.4515172471, 'transitions': 105}, {'sum': 144498.1929526834, 'transitions': 106}, {'sum': 318.43602180480957, 'transitions': 107}, {'sum': 2079344.0997261645, 'transitions': 201}, {'sum': 2163357.6385990065, 'transitions': 202}, {'sum': 1044572.4286110598, 'transitions': 203}, {'sum': 2005388.0396160348, 'transitions': 204}, {'sum': 467534.09239385265, 'transitions': 205}, {'sum': 69441.04652956047, 'transitions': 206}, {'sum': 204.7110824584961, 'transitions': 207}, {'sum': 541605.4405773312, 'transitions': 301}, {'sum': 792680.7531657424, 'transitions': 302}, {'sum': 872157.0463266934, 'transitions': 303}, {'sum': 384453.5201430526, 'transitions': 304}, {'sum': 33053.113399131624, 'transitions': 305}, {'sum': 9084.077752767826, 'transitions': 306}, {'sum': 272.9516658782959, 'tr

#### Exporting the change mask to Drive and Asset

In [18]:
# converting the FeatureCollection to Geometry for export
aoi_geom = aoi_windhoek.geometry()

# exporting to Google drive with GEE API
export_change_mask = ee.batch.Export.image.toDrive(**{
    'image': changed_areas_2015_2017,
    'description': 'change_mask_2015_2017', # TODO: change name here
    'folder': 'masterthesis/change_detection_results', # TODO: change name here
    'scale': 4.77,
    'crs': 'EPSG:32733',
    'region': aoi_geom
})

# starting the process
export_change_mask.start()

# tracking the process
while export_change_mask.active():
  print('Polling for task (id: {}).'.format(export_change_mask.id))
  time.sleep(5)

Polling for task (id: VGHKYC5DVCNYJHFRRG3DZL4S).
Polling for task (id: VGHKYC5DVCNYJHFRRG3DZL4S).
Polling for task (id: VGHKYC5DVCNYJHFRRG3DZL4S).
Polling for task (id: VGHKYC5DVCNYJHFRRG3DZL4S).
Polling for task (id: VGHKYC5DVCNYJHFRRG3DZL4S).
Polling for task (id: VGHKYC5DVCNYJHFRRG3DZL4S).
Polling for task (id: VGHKYC5DVCNYJHFRRG3DZL4S).
Polling for task (id: VGHKYC5DVCNYJHFRRG3DZL4S).
Polling for task (id: VGHKYC5DVCNYJHFRRG3DZL4S).
Polling for task (id: VGHKYC5DVCNYJHFRRG3DZL4S).
Polling for task (id: VGHKYC5DVCNYJHFRRG3DZL4S).
Polling for task (id: VGHKYC5DVCNYJHFRRG3DZL4S).
Polling for task (id: VGHKYC5DVCNYJHFRRG3DZL4S).
Polling for task (id: VGHKYC5DVCNYJHFRRG3DZL4S).
Polling for task (id: VGHKYC5DVCNYJHFRRG3DZL4S).
Polling for task (id: VGHKYC5DVCNYJHFRRG3DZL4S).
Polling for task (id: VGHKYC5DVCNYJHFRRG3DZL4S).
Polling for task (id: VGHKYC5DVCNYJHFRRG3DZL4S).
Polling for task (id: VGHKYC5DVCNYJHFRRG3DZL4S).
Polling for task (id: VGHKYC5DVCNYJHFRRG3DZL4S).
Polling for task (id

In [ ]:
# exporting to Google Asset
export_change_mask = ee.batch.Export.image.toAsset(**{
  'image': changed_areas_2015_2017,
  'description': 'Export Change Mask map',
  'assetId': 'users/s85315/masterthesis/change_detection_results/change_mask_2015_2017', # TODO: change name here
  'scale': 4.77,
  'crs': 'EPSG:32733',
  'region': aoi_geom
})

# starting the process
export_change_mask.start()

# tracking the process
while export_change_mask.active():
  print('Polling for task (id: {}).'.format(export_change_mask.id))

In [30]:
# zones = changed_areas_2015_2017.select('classification').eq(1)

# vectors = zones.reduceToVectors(**{
#   'geometry': aoi_geom,
#   'scale': 4.77,
#   'reducer': ee.Reducer.mean()
# })

# display = ee.Image(0).updateMask(0).paint(vectors, '000000', 1)

# visParams2010 = {'min':0, 'max':1, 'palette': ['white', 'red']};


# Map.addLayer(display, {}, 'Vectors')

#### 3.2 Two-Step Accuracy Assessment

#### 3.3 Creating visualization

### 3 CD for TOI 2015 to 2017

#### 3.1 Performing the CD analysis

#### 3.2 Two-Step Accuracy Assessment

### 3 CD for TOI 2015 to 2017

#### 3.1 Performing the CD analysis

#### 3.2 Two-Step Accuracy Assessment

##### Resources for code snippets

https://colab.research.google.com/github/csaybar/EEwPython/blob/dev/10_Export.ipynb \
https://worldbank.github.io/OpenNightLights/tutorials/mod6_6_RF_classifier.html \
https://towardsdatascience.com/how-to-easily-create-tables-in-python-2eaea447d8fd \
https://developers.google.com/earth-engine/apidocs/ee-classifier-smilerandomforest